# Importação e Tratamento de Dados

In [1]:
import numpy as np
import pandas as pd
import sklearn_helper as skh
import os

In [2]:
# Link em que se encontram os dados a serem utilizados na análise
LINK_DADOS = "https://drive.google.com/file/d/1cXtMwZsm3Eo5NejaCYjVgA1U46It61r5/view"
PASTA_DADOS = "Dados/"

TAXA_DE_AMOSTRAGEM_HZ = 208

# Configuração da função find_peaks
COLUNA_PICOS = 'imu1accz'

DISTANCIA_ENTRE_PICOS_S = 8 * TAXA_DE_AMOSTRAGEM_HZ
ALTURA_MINIMA_PICO = 5

# Configuração do recorte de dados no entorno de cada pico
COMPRIMENTO_DA_LEITURA_S = int(1 * TAXA_DE_AMOSTRAGEM_HZ)
OFFSET_ANTES_DO_PICO_S = int(.1 * TAXA_DE_AMOSTRAGEM_HZ)

COLUNAS_A_REMOVER = ['time', 'dac1', 'dac2', 'dac3', 'dac4', 'log']

LABELS = {
    'metalico':     0,
    'amortecido':   1
}

In [3]:
skh.baixar_dados(LINK_DADOS, output_dir=PASTA_DADOS)

False

In [4]:
skh.unzip_arquivos_da_pasta(PASTA_DADOS)

In [5]:
feathers_gabriel = [PASTA_DADOS + path for path in os.listdir(PASTA_DADOS) if path.find('Gabriel') != -1]
feathers_gabriel


['Dados/Dados_Impacto_Metalico_Gabriel.feather',
 'Dados/Dados_Impacto_Amortecido_Gabriel.feather']

In [6]:
feathers_joao = [PASTA_DADOS + path for path in os.listdir(PASTA_DADOS) if path.find('Joao') != -1]
feathers_joao

['Dados/Impacto_Metalico_Joao_2.feather',
 'Dados/Impacto_Amortecido_Joao_1.feather',
 'Dados/Impacto_Amortecido_Joao_2.feather',
 'Dados/Impacto_Metalico_Joao_1.feather']

In [7]:
def importar_e_preparar(feathers_paths: str) -> pd.DataFrame:
    """
    Retorna um data frame onde cada linha contém as informações achatadas de um período de dados.
    Importa uma lista de caminhos para arquivos feather, recortando os períodos de dados e atribuindo labels.
    
    Notas
    ----------
    * A preparação ocorre conforme as constantes definidas no ínicio do código.
    * As labels são encontradas automaticamente, comparando chaves fornecidas em LABELS com os nomes dos arquivos.
    """    
    
    dfs = []
    for path in feathers_paths:
        # Importação
        df = pd.read_feather(path).drop(COLUNAS_A_REMOVER, axis= 1)
        dados = skh.DadosVibracao(df, COLUNA_PICOS, ALTURA_MINIMA_PICO, DISTANCIA_ENTRE_PICOS_S)
        dados_achatados = skh.achatar_dados(dados[:, COMPRIMENTO_DA_LEITURA_S, OFFSET_ANTES_DO_PICO_S])
        
        # Atribuição de categoria
        label_key = (path.lower().split("_") & LABELS.keys()).pop()
        dados_achatados['label'] = LABELS[label_key]
        dfs.append(dados_achatados)
    
    out_df = pd.concat(dfs).dropna().reset_index().drop('index', axis=1)

    return out_df

In [8]:
df_preparado_gabriel = importar_e_preparar(feathers_gabriel)
df_preparado_joao = importar_e_preparar(feathers_joao)
            

In [9]:
df_preparado_gabriel_joao = pd.concat([df_preparado_gabriel, df_preparado_joao])
df_preparado_gabriel_joao


,imu1accx_0,imu1accy_0,imu1accz_0,imu1gyrox_0,imu1gyroy_0,imu1gyroz_0,imu2accx_0,imu2accy_0,imu2accz_0,imu2gyrox_0,...,imu1gyrox_187,imu1gyroy_187,imu1gyroz_187,imu2accx_187,imu2accy_187,imu2accz_187,imu2gyrox_187,imu2gyroy_187,imu2gyroz_187,label
0,-0.120907,19.540277,0.051475,-2.780914,-3.707886,-3.540039,-2.236184,18.770541,-11.256339,-3.505707,...,-1.071930,-3.791809,-3.738403,-1.738190,19.139248,-5.266047,-4.230499,-7.247925,-2.262115,0
1,-0.381875,19.733010,-1.618480,0.556946,-3.715515,-3.742218,-1.140837,18.775329,-1.022324,1.987457,...,0.610352,-3.765106,-3.578186,-1.770512,19.592949,1.713051,0.663757,-6.900787,-2.468109,0
2,-2.000355,20.728998,-6.025008,2.937317,-2.944946,-0.930786,-5.759252,18.312051,-1.217452,-0.476837,...,2.464294,-3.887177,-3.726959,-2.064999,19.068619,-0.500388,-0.656128,-6.965637,-2.399445,0
3,-1.574188,18.280927,9.489418,5.214691,-3.978729,-3.383636,-2.784457,19.914969,-4.699818,1.956940,...,3.822327,-3.726959,-3.700256,-2.085350,19.796456,-1.892616,4.585266,-6.637573,-2.437592,0
4,-1.558625,20.047848,4.432864,-1.132965,-3.738403,5.077362,-1.000776,14.401123,-12.340912,-9.006500,...,0.251770,-3.627777,-3.688812,-2.103306,18.825608,-2.163161,-2.956390,-7.122040,-2.307892,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,-0.144849,20.488381,7.829039,-1.377106,-2.857208,-3.276825,-11.119870,18.338388,2.700659,9.880066,...,1.586914,-3.284454,-4.577637,-1.171962,20.289662,5.607220,6.607056,-6.229401,-2.674103,0
183,0.084994,21.795615,-3.593697,-2.571106,-3.536224,-4.295349,-3.724181,16.765397,-2.155979,4.688263,...,1.567841,-3.803253,-4.631042,-1.568202,19.887436,2.043451,3.234863,-6.191254,-2.433777,0
184,1.589750,22.094890,1.821987,-1.308441,-0.946045,-2.059937,-20.321984,18.899828,2.677915,2.471924,...,-0.869751,-3.463745,-4.489899,-1.150414,20.039468,1.732205,-1.525879,-7.083893,-2.494812,0
185,2.197877,22.892159,-7.529764,0.644684,-2.876282,-2.105713,-3.434482,14.748282,-2.619257,16.090393,...,2.201080,-3.322601,-4.444122,-1.219846,20.563798,7.773973,10.478973,-5.313873,-2.754211,0


# Scikit Learn - Random Forest

In [10]:
from sklearn.ensemble import RandomForestClassifier

## IA Gabriel

In [11]:
x_gabriel = df_preparado_gabriel.drop('label', axis=1)
y_gabriel = df_preparado_gabriel['label']

rf_gabriel = RandomForestClassifier()

In [12]:
skh.testar_precisao(rf_gabriel, x_gabriel, y_gabriel)


    Precisão Média:   92.86%
    Desvio Padrão:    0.04
    Variância:        1.47e-03
    


## IA Joao

In [13]:
x_joao = df_preparado_joao.drop('label', axis=1)
y_joao = df_preparado_joao['label']

rf_joao = RandomForestClassifier()

In [14]:
skh.testar_precisao(rf_joao, x_joao, y_joao)


    Precisão Média:   95.32%
    Desvio Padrão:    0.04
    Variância:        1.88e-03
    


## Teste Cruzado

In [15]:
# IA treinada com dados de Gabriel testada nos dados de João
skh.testar_precisao_dados_fixos(rf_gabriel, [x_gabriel, y_gabriel], [x_joao, y_joao])

  
    Precisão Média:   60.96%
    Desvio Padrão:    0.03
    Variância:        7.32e-04
    


In [16]:
# IA treinada com dados de João testada nos dados de Gabriel
skh.testar_precisao_dados_fixos(rf_joao, [x_joao, y_joao], [x_gabriel, y_gabriel])

  
    Precisão Média:   70.37%
    Desvio Padrão:    0.08
    Variância:        6.28e-03
    


## IA com todos os dados

In [17]:
x = df_preparado_gabriel_joao.drop('label', axis=1)
y = df_preparado_gabriel_joao['label']

rf = RandomForestClassifier()

In [18]:
skh.testar_precisao(rf, x, y)


    Precisão Média:   93.24%
    Desvio Padrão:    0.03
    Variância:        8.30e-04
    
